In [ ]:
import os
import itertools
import glob
from PIL import Image
import numpy as np
from matplotlib import pyplot as plt
from facenet_pytorch import MTCNN, InceptionResnetV1
import clusteringv15 as myf

print("Setting up detection pipeline.\n")

# If required, create a face detection pipeline using MTCNN:
mtcnn = MTCNN(image_size=160, margin=0)

# Create an inception resnet (in eval mode):
resnet = InceptionResnetV1(pretrained='vggface2').eval()

In [ ]:
image_name = '13.jpg'

input_ = Image.open('./data/input_data/' + image_name)
input_cropped = mtcnn(input_,save_path='./data/input_data_aligned/' + image_name)
input_embedding = resnet(input_cropped.unsqueeze(0))

input_small = Image.open('./data/input_data_aligned/' + image_name)
fig, axarray = plt.subplots(1,2,figsize=(8, 6))
axarray[0].imshow(input_)
axarray[0].axis('off')
axarray[1].imshow(input_small)
axarray[1].axis('off')
fig.show()

In [ ]:
# Making sure all necessary directories exists
from pathlib import Path
Path("./data/input_data_aligned").mkdir(parents=True, exist_ok=True)
Path("./data/temp/heatmaps").mkdir(parents=True, exist_ok=True)
Path("./data/temp/boxes").mkdir(parents=True, exist_ok=True)
Path("./data/temp/noise").mkdir(parents=True, exist_ok=True)
score_arrays=[]

In [ ]:
img_path = './data/input_data_aligned/'+image_name
img = Image.open(img_path)

denominators = [2,4,5,8]
noise_factors = np.arange(0,1,0.1)

#clear heatmap datapath
print("Clear heatmap_savepath.")
heatmap_path = './data/temp/heatmaps/'
heatmap_data = list(itertools.chain(*(glob.glob(heatmap_path + '*.%s' % ext) for ext in ["jpg", "jpeg", "png"])))
for f in heatmap_data:
    os.remove(f)

In [ ]:
#get landmarks
boxes, probs, landmarks = mtcnn.detect(input_, landmarks=True)
eyes, nose, mouth = myf.get_feature_bboxes(img, boxes, landmarks,save_path='./data/temp/landmarks/')

In [ ]:
#model eyes
list_score_arrays = {}
chop_path = './data/temp/landmarks/'
noise_path = './data/temp/noise/'
for nom in denominators:
    chops = myf.cluster_face(img,denominator=nom,save_path=chop_path)
    new_comps = myf.cluster_embeddings(img,noise_factors,chop_path,chops,mtcnn,resnet,save_path=noise_path)

In [ ]:
new_comps